In [1]:
# ----------------------------------------------------------------------------------------- #
# STEP01_v1
    # A. Detect Spontaneous Electrode Saturation ( SES )
    # B. Repairing Spontaneous Electrode Saturation
    # C. Rebuild discarded channels ( Empties )
    # D. Save repaired segments
    # E. Plot repaired segments
# ----------------------------------------------------------------------------------------- #
# * This value can or should be modified
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Custom made functions
# ----------------------------------------------------------------------------------------- #
# Path to the folder where the custom functions are storage
PATHMODULE = joinpath( homedir( ), "Dropbox", "CodigoGithub" ); #                           *
push!( LOAD_PATH, PATHMODULE );
using STEP01_v092024
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Julia's native packages
# ----------------------------------------------------------------------------------------- #
using JLD2
using StatsBase
using Plots
using Measures
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Searching for directories and files to use
# ----------------------------------------------------------------------------------------- #
PATHSTART = joinpath( homedir( ), "Desktop" );
PATHSINFO, _ = FindDirsFiles( PATHSTART, "Info" );
for i in 1:length( PATHSINFO )
    f = PATHSINFO[ i ];
    println( string( "$i - $f" ) );
end
Infos = length( PATHSINFO );
# ----------------------------------------------------------------------------------------- #
# Select a file to work with from now on.
# ----------------------------------------------------------------------------------------- #
info = 0;
while info > Infos || info < 1
    print( "Enter the file number: [ between 1 and $Infos ]" );
    info = parse( Int, readline( ) );
end
PATHINFO = PATHSINFO[ info ];
println( "\nPath selected: $PATHINFO" ); 
# ----------------------------------------------------------------------------------------- #

[ Info: Precompiling STEP01_v092024 [top-level]
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
[ Info: Skipping precompilation since __precompile__(false). Importing STEP01_v092024 [top-level].
┌ Warning: Replacing docs for `STEP01_v092024.ms2frs :: Tuple{Real, Real}` in module `STEP01_v092024`
└ @ Base.Docs docs/Docs.jl:243


1 - /home/isabel/Desktop/Data/060915s01/Info
2 - /home/isabel/Desktop/Data/700_3-3/Info
3 - /home/isabel/Desktop/Data/lesion_parakarel_140116/Info
4 - /home/isabel/Desktop/GarbageCollector/lesion_parakarel_140116/Info
Enter the file number: [ between 1 and 4 ]stdin> 3

Path selected: /home/isabel/Desktop/Data/lesion_parakarel_140116/Info


In [2]:
# ----------------------------------------------------------------------------------------- #
# Generating the new folders and files
# ----------------------------------------------------------------------------------------- #
PATHMAIN = dirname( PATHINFO );
PATHSTEP00 = joinpath( PATHMAIN, "STEP00" );
PATHSTEP01 = replace( PATHSTEP00, "STEP00" => "STEP01" ); mkpath( PATHSTEP01 );
PATHFIGURES = joinpath( PATHMAIN, "Figures" );
PATHFIGURES_STEP01 = joinpath( PATHFIGURES, "STEP01" ); mkpath( PATHFIGURES_STEP01 );
PATHFIGURES_GENERAL = joinpath( PATHFIGURES, "GENERAL" ); mkpath( PATHFIGURES_GENERAL );
#
FILESTEP01 = joinpath( PATHMAIN, "Info", "STEP01.jld2" );
FILEVARIABLES = joinpath( PATHINFO, "Variables.jld2" );
FILESTEP00 = joinpath( PATHMAIN, "Info", "STEP00.jld2" );
FILEPARAMETERS = joinpath( PATHINFO, "Parameters.jld2" );
FILESVOLTAGE = SearchDir( PATHSTEP00, ".jld2" );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Extracting the variables contained in the brw file
# ----------------------------------------------------------------------------------------- #
Variables = LoadDict( FILEVARIABLES );
step00 = LoadDict( FILESTEP00 );
Parameters = LoadDict( FILEPARAMETERS );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Constants
# ----------------------------------------------------------------------------------------- #
#= 
    All of the following values fall into the * category, 
    meaning that they can or should be modified
=#
# ----------------------------------------------------------------------------------------- #
Δt = Parameters[ "Δt" ]; # ms for "Voltage Shift Deviation"
N = Parameters[ "N" ];
cm_ = Parameters[ "cm_" ]; # Color map for all the heatmap plots
# μV of range that are assigned to the maximum voltage to set the threshold for saturation.
cte = 100; 
MaxVolt = Variables[ "MaxVolt" ]; # Maximum possible voltage registered by the equipment
THR_SES = abs( MaxVolt - cte ); # Threshold to be considered for setting saturation
# Font type for all the plots
plotfonts = Plots.font( pointsize = 10, family = "sans-serif" );
minchan = 8; # minimum channels for channel reconstruction                            
maxrad = 4; # maximum ratio of neighborhood, for channel reconstruction
maxIt = 5; # maximum number of iterations, for channel reconstruction
n0s = length( string( N ) );
fc = :royalblue3; # Color map for the bar plot
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
Empties = step00[ "Empties" ]; # List of previously discarded channels
# ----------------------------------------------------------------------------------------- #

In [3]:
# ----------------------------------------------------------------------------------------- #
# Evaluation of individual bin behavior
# ----------------------------------------------------------------------------------------- #
aux = vcat( sum.( step00[ "Cardinality" ], dims = 1 )... );
W = zscore( aux );
t = "\n" ^ 1 * "Raw Bin Behavior";
xl = "n bin";
yl = "zscore of Σ( Cardinality )";
P0 = BarPlot( W, fc, t, xl, yl );
P0 = plot( 
    P0, 
    plotfont = plotfonts,
    xticks =  ( 0:5:N )
);
FILEFIGURE = joinpath( PATHFIGURES_GENERAL,"RawBinBehavior" );
Plots.png( P0, FILEFIGURE );
Plots.svg( P0, FILEFIGURE );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Pre-allocating the arrays to store the results
# ----------------------------------------------------------------------------------------- #
Cardinality = Array{ Any }( undef, N ); fill!( Cardinality, [ ] );
VoltageShiftDeviation = Array{ Any }( undef, N ); fill!( VoltageShiftDeviation, [ ] );
Sats = Array{ Any }( undef, N ); fill!( Sats, [ ] );
Repaired = Array{ Any }( undef, N ); fill!( Repaired, [ ] );
# ----------------------------------------------------------------------------------------- #

# ------------------------------------------------------------------------------------- #
for n in 1:N
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    BINNAME = joinpath( PATHSTEP00, string( "BIN", lpad( n, n0s, "0" ), ".jld2" ) );
    BINRAW = Float64.( LoadDict( BINNAME ) ); # Load the n-segment in Float64
    nChs, nFrs = size( BINRAW );
    BINPATCH = deepcopy( BINRAW );
    BINPATCH[ Empties, : ] .= 0; # Discarded channels are flattened to 0
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # A. Detect Spontaneous Electrode Saturation ( SES )
    # ------------------------------------------------------------------------------------- #
    SatChs, SatFrs = SupThr( BINRAW, THR_SES );
    # Remove empty channels from the list to properlly evaluate saturations ( not needed )
    aux = SatChs .∉ [ Empties ];
    SatChs = SatChs[ aux ];
    SatFrs = SatFrs[ aux ];
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    Chs4Repair = [ ];
    Frs4Repair = [ ];
    for ch in 1:length( SatChs )
        sch = SatChs[ ch ];
        sfr = SatFrs[ ch ];
        g = ReduceArrayDistance( sfr, 1 );
        for G in g
            push!( Chs4Repair, sch );
            push!( Frs4Repair, G );
        end
    end
    OriginalSaturations = Dict(
        "Chs" => Chs4Repair,
        "Frs" => Frs4Repair
    );
    #= Saturations due to amplifier malfunction, electrode damage, or external stimuli such 
    as saturation by light, electrical noise or bubbles.=#
    Sats[ n ] = OriginalSaturations; 
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # B. Repairing spontaneous saturated segments
    # ------------------------------------------------------------------------------------- #
    FRS = OriginalSaturations[ "Frs" ];
    CHS = OriginalSaturations[ "Chs" ];
    for s = 1:length( Chs4Repair )
        ch = Chs4Repair[ s ];
        fr = Frs4Repair[ s ];
        NoFrs = sort( vcat( FRS[ CHS .∈ [ ch ] ] ...) );
        nsf = length( fr );
        ValidFrames = setdiff( 1:nFrs, NoFrs );
        fictional_segment = sample( ValidFrames, nsf );
        channel = BINPATCH[ ch, : ];
        fictional_segment = channel[ fictional_segment ];
        BINPATCH[ ch, fr ] = fictional_segment;
    end
    Repaired[ n ] = Dict( 
        "Chs" => Chs4Repair,
        "Frs" => Frs4Repair
    );
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # C. Rebuild discarded channels ( Empties )
    # ------------------------------------------------------------------------------------- #
    for emptie in Empties
        rad = 1
        _, neigh = Neighbours( emptie, rad );
        while length( neigh ) <= minchan && rad <= maxrad
            rad = rad + 1;
            _, neigh = Neighbours( emptie, rad );
        end
        neighs = sort( sample( neigh, minchan, replace = false ) );
        BINNEIGHS = BINPATCH[ neighs, : ];
        fictional_channel = ReconstructChannels( BINNEIGHS, maxIt );
        BINPATCH[ emptie, : ] = fictional_channel;
    end
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # D. Save repaired segments
    # ------------------------------------------------------------------------------------- #
    jldsave( replace( BINNAME, "STEP00" => "STEP01" ); Data = Float16.( BINPATCH ) );
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # Second Evaluation, Cardinality and Voltage Shift Deviation
    # ------------------------------------------------------------------------------------- #
    CAR = [ ];
    VSD = [ ];
    CAR = UniqueCount( BINPATCH );
    VSD = STDΔV( Variables, BINPATCH, Δt );
    SecondEvaluation = Dict(
        "Cardinality"            => CAR,
        "VoltageShiftDeviation"  => VSD,
    );
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    Cardinality[ n ] = SecondEvaluation[ "Cardinality" ];
    VoltageShiftDeviation[ n ] = SecondEvaluation[ "VoltageShiftDeviation" ];
    # ------------------------------------------------------------------------------------- #

    # ------------------------------------------------------------------------------------- #
    # E. Second graph, Evaluation of the repaired segment.
    # ------------------------------------------------------------------------------------- #
    aux = PatchEmpties( CAR, Empties );
    P0 = plot( );
    P0 = Zplot( zscore( aux ), cm_, "\n" ^ 2 * "Cardinality of the Voltage" );
    aux = PatchEmpties( VSD, Empties );
    P1 = plot( );
    P1 = Zplot( zscore( aux ), cm_, "\n" ^ 2 * "Voltage Shift Deviation" );
    P = plot( );
    P = plot(
        P0, P1,
        layout = ( 1, 2 ),
        wsize = ( 800, 400 )
        );
    title = plot( );
    title = plot(
        title = "\n" ^ 2 * "Second evaluation, Repaired Data",
        grid = false,
        showaxis = false,
        bottom_margin = -50Plots.px
        );
    F = plot( );
    F = plot( 
        title, P,
        layout = @layout( [ A{ 0.1h }; B{ 0.9h } ] ),
        wsize = ( 800, 500 ),
        titlefont = plotfonts,
    );
    FILEFIGURE = joinpath( PATHFIGURES_STEP01, string( "BIN", lpad( n, n0s, "0" ) ) );
    Plots.png( F, FILEFIGURE );
    # ------------------------------------------------------------------------------------- #
end

# ----------------------------------------------------------------------------------------- #
# Saving the results of all segments
# ----------------------------------------------------------------------------------------- #
step01 = Dict( 
    "Cardinality"           => Cardinality,
    "VoltageShiftDeviation" => VoltageShiftDeviation,
    "Sats"                  => Sats,
    "Repaired"              => Repaired,
    "Empties"               => Empties
    );
jldsave( FILESTEP01; Data = step01 );
# ----------------------------------------------------------------------------------------- #

# ----------------------------------------------------------------------------------------- #
# Saving the parameters for step-01
# ----------------------------------------------------------------------------------------- #
NewParameters = Dict(
    "THR_SES" => THR_SES,
    "minchan" => minchan,
    "maxrad"  => maxrad,
    "maxIt"   => maxIt,
);
Parameters = merge( Parameters, NewParameters );
jldsave( FILEPARAMETERS; Data = Parameters );
# ----------------------------------------------------------------------------------------- #
# end of the step-01
# ----------------------------------------------------------------------------------------- #